In [1]:
from fastapi import FastAPI, Query
from fastapi.middleware.cors import CORSMiddleware
import requests

app = FastAPI()
app.add_middleware(
    CORSMiddleware,
    allow_origins=["*"],
    allow_credentials=True,
    allow_methods=["*"],
    allow_headers=["*"],
)
# Replace with your actual API key
API_URL="https://weather.visualcrossing.com/VisualCrossingWebServices/rest/services/timeline/silchar/today?unitGroup=us&key=4RH3KGN9E38EDN8W7XJT42SQ5&contentType=json"
@app.get("/weather/")
def get_weather(location: str = Query(..., description="City or location name")):
    url = f"https://weather.visualcrossing.com/VisualCrossingWebServices/rest/services/timeline/{location}/today?unitGroup=us&key=4RH3KGN9E38EDN8W7XJT42SQ5&contentType=json"


    response = requests.get(url)

    if response.status_code != 200:
        return {"error": "Failed to fetch weather data"}

    data = response.json()
    return data['days']


In [2]:
import pandas as pd
data=pd.DataFrame(get_weather(location="guwahati"))

In [3]:
data.shape

(1, 37)

In [4]:
def preprocess_data(weather_data):
    # Convert datetime to features
    data['datetime'] = pd.to_datetime(data['datetime'])
    data['year'] = data['datetime'].dt.year
    data['month'] = data['datetime'].dt.month
    data['day'] = data['datetime'].dt.day
    data['day_of_year'] = data['datetime'].dt.dayofyear

    # Extract time components from sunrise and sunset
    data['sunrise_hour'] = pd.to_datetime(data['sunrise']).dt.hour
    data['sunrise_minute'] = pd.to_datetime(data['sunrise']).dt.minute
    data['sunset_hour'] = pd.to_datetime(data['sunset']).dt.hour
    data['sunset_minute'] = pd.to_datetime(data['sunset']).dt.minute
    data['daylight_hours'] = (pd.to_datetime(data['sunset']) - pd.to_datetime(data['sunrise'])).dt.total_seconds()/3600

    # Handle categorical variables (conditions)
    conditions_dummies = pd.get_dummies(data['conditions'], prefix='cond')
    data = pd.concat([data, conditions_dummies], axis=1)

    # Drop unnecessary columns
    drop_cols = ['name', 'datetime', 'sunrise', 'sunset', 'conditions', 'description', 
             'icon', 'stations', 'preciptype', 'severerisk']
    data = data.drop(columns=drop_cols)
    return dATA 

In [5]:
data = preprocess_data(data)

In [6]:
data.shape

(1, 43)

In [7]:
# Define features and target
X = data.drop(columns=['solarenergy'])
y = data['solarenergy']

LOADING SCALER

In [8]:
import pickle
with open("xgboost_scaler.pkl", "rb") as f:
    scaler = pickle.load(f)
X_scaled = scaler.transform(X)

ValueError: The feature names should match those that were passed during fit.
Feature names unseen at fit time:
- cond_Rain
- datetimeEpoch
- hours
- pressure
- source
- ...
Feature names seen at fit time, yet now missing:
- sealevelpressure
